# Joining All Data & Tableau Preparation

This notebook is intended for joining all the processed data from previous notebook,mold them into shapes that fit into one table for a join, and load into Google Sheet for a Tableau Public Visualization.

P.S. Checkout previous notebooks for more detailed data processing

In [1]:
# Importing libraries
import glob  #using glob to get all the (forJoins) csv files
import pandas as pd
from datetime import datetime

In [4]:
# Reading in all the data together
dfs = []
for i in glob.glob("FlatData/(forJoins)*"):
    print(i)
    df = pd.read_csv(i)
    dfs.append(df)

FlatData\(forJoins)covid19_tweets.csv
FlatData\(forJoins)CumCovidCases.csv
FlatData\(forJoins)DailyCovidCases.csv
FlatData\(forJoins)ZillowPropertyValueData.csv


## Shaping the Daily Covid Data for Tableau

In [18]:
CovidDf = dfs[2].copy()
CovidDf_cum = dfs[1].copy()

In [19]:
meltCol = []
for i in range(1, len(CovidDf.columns)):
    meltCol.append(CovidDf.columns[i])

In [20]:
# Trimming Date range
CovidDf = CovidDf.rename(columns={'Unnamed: 0':'date'})
CovidDf['date'] = pd.to_datetime(CovidDf['date'])

CovidDf = CovidDf[CovidDf['date'] >= datetime(2020,2,1)]



CovidDf_cum = CovidDf_cum.rename(columns={'Unnamed: 0':'date'})
CovidDf_cum['date'] = pd.to_datetime(CovidDf_cum['date'])

CovidDf_cum = CovidDf_cum[CovidDf_cum['date'] >= datetime(2020,2,1)]

In [27]:
# Getting frames of cumulative cases and daily cases ready

meltedDailyCases1 = pd.melt(CovidDf, 
                            id_vars=['date'], 
                            value_vars=meltCol)

meltedDailyCases2 = pd.melt(CovidDf_cum, 
                            id_vars=['date'], 
                            value_vars=meltCol)

In [29]:
meltedDailyCases = pd.merge(meltedDailyCases1,
                            meltedDailyCases2,
                            left_on=['date', 'variable'],
                            right_on=['date', 'variable'])

In [30]:
meltedDailyCases = meltedDailyCases.rename(columns={"variable":"State",
                                                    "value_x":"DailyCases",
                                                    "value_y":"TotalCases"})

meltedDailyCases

,date,State,DailyCases,TotalCases
0,2020-02-01,Alabama,0.0,0
1,2020-02-02,Alabama,0.0,0
2,2020-02-03,Alabama,0.0,0
3,2020-02-04,Alabama,0.0,0
4,2020-02-05,Alabama,0.0,0
...,...,...,...,...
12059,2020-08-22,Wyoming,19.0,3543
12060,2020-08-23,Wyoming,35.0,3578
12061,2020-08-24,Wyoming,25.0,3603
12062,2020-08-25,Wyoming,29.0,3632


## Shaping the Real Estates Prices for Tableau

In [31]:
RealEstateDf = dfs[3].copy()

In [32]:
RealEstateDf = RealEstateDf.rename(columns={'Unnamed: 0':'date'})
RealEstateDf['date'] = pd.to_datetime(RealEstateDf['date'])

RealEstateDf = RealEstateDf[RealEstateDf['date'] >= datetime(2020,2,1)]

In [33]:
meltCol2 = []
for i in range(1, len(RealEstateDf.columns)):
    meltCol2.append(RealEstateDf.columns[i])

In [34]:
meltedRealEstates = pd.melt(RealEstateDf, 
                            id_vars=['date'], 
                            value_vars=meltCol2)

In [35]:
meltedRealEstates = meltedRealEstates.rename(columns={"variable":"State",
                                                      "value":"RealEstateValues"})

meltedRealEstates

,date,State,RealEstateValues
0,2020-02-29,Alabama,130623.047619
1,2020-03-31,Alabama,131203.380952
2,2020-04-30,Alabama,131796.857143
3,2020-05-31,Alabama,132456.285714
4,2020-06-30,Alabama,133160.000000
...,...,...,...
301,2020-03-31,Wyoming,297820.888889
302,2020-04-30,Wyoming,298070.888889
303,2020-05-31,Wyoming,298086.222222
304,2020-06-30,Wyoming,298084.222222


## Shaping the Covid Sentiment for Tableau

In [36]:
CovidSentiment = dfs[0].copy()
CovidSentiment['VaderRange'] = CovidSentiment['VaderMax'] - CovidSentiment['VaderMin']

# This shows how extreme people's sentiments are during the pandemic
# Most people have strong reaction, and the sentiment is very polarized
CovidSentiment['VaderRange'].describe()

count    24.000000
mean      1.944725
std       0.025244
min       1.879900
25%       1.935350
50%       1.946400
75%       1.966925
max       1.980900
Name: VaderRange, dtype: float64

In [37]:
CovidSentiment['date'] = pd.to_datetime(CovidSentiment['date'])

## Pulling Stock Data For Tableau

In [38]:
import sys
sys.path.append("..")

from DataCollection import StockDataCollection

In [39]:
# Getting the API key from txt file
with open('../DataCollection/SecretKeys/FreeAlphavantageKey.txt', 'r') as file:
    key = file.read().replace('\n', '')

In [40]:
StockToLookAt = "SPY"

Connector = StockDataCollection.ConnectAPI(key=key)
SP500 = StockDataCollection.DataHandle.DataTrimmer(Connector.LoadingData(StockToLookAt, FullHistory=True))

In [41]:
StockData = pd.DataFrame(SP500)
StockData = StockData.reset_index()
StockData = StockData.rename(columns={'SPY':'Stock Market', 'index':'date'})
StockData = StockData[StockData['date'] >= datetime(2020,2,1)]

In [42]:
StockData

,date,Stock Market
21,2020-02-03,324.12
22,2020-02-04,329.06
23,2020-02-05,332.86
24,2020-02-06,333.98
25,2020-02-07,332.20
...,...,...
161,2020-08-21,339.48
162,2020-08-24,342.92
163,2020-08-25,344.12
164,2020-08-26,347.57


## Joining The Data

In [46]:
# Joining Daily cases and Real Estates price data by date and Location
# Using a left join because our goal is to study covid related movements

JoinDf = pd.merge(meltedDailyCases, 
                  meltedRealEstates, 
                  how='left', 
                  left_on=['date','State'],
                  right_on=['date','State'])


# Since we only have monthly Real Estate Price data (update during the end of a month)
# We will duplicate the value to fill the entire month

JoinDf['RealEstateValues'] = JoinDf['RealEstateValues'].fillna(method='backfill')

In [47]:
# Joining Stock Market index data with the first join output above
JoinDf2 = pd.merge(JoinDf,
                   StockData,
                   how='left',
                   left_on='date',
                   right_on='date')

In [48]:
# Filling in the weekends because the market is closed for the weekend
JoinDf2['Stock Market'] = JoinDf2['Stock Market'].fillna(method='ffill')
JoinDf2['Stock Market'] = JoinDf2['Stock Market'].fillna(method='bfill')

In [49]:
# Joining the final piece of Social Sentiment data
OutputJoin = pd.merge(JoinDf2,
                      CovidSentiment,
                      how='left',
                      left_on='date',
                      right_on='date')

In [52]:
# Getting values columns for cleaning so that we won't potentially mess up the others
CleaningUseCol = CovidSentiment.columns[1:].tolist()

In [53]:
# We use 0 to fill na here to show days that no one uses the hashtag covid19
# Due to the unfinished nature of language subject data, there are days that
# appeared to have no one tweeted about covid19
# More validation is need for the Kaggle Data Source engineer

OutputJoin[CleaningUseCol] = OutputJoin[CleaningUseCol].fillna(0)

In [54]:
OutputJoin['date'] = OutputJoin['date'].astype(str)

In [55]:
OutputJoin.head()

,date,State,DailyCases,TotalCases,RealEstateValues,Stock Market,VaderMin,VaderMax,VaderMean,VaderRange
0,2020-02-01,Alabama,0.0,0,130623.047619,324.12,0.0,0.0,0.0,0.0
1,2020-02-02,Alabama,0.0,0,130623.047619,324.12,0.0,0.0,0.0,0.0
2,2020-02-03,Alabama,0.0,0,130623.047619,324.12,0.0,0.0,0.0,0.0
3,2020-02-04,Alabama,0.0,0,130623.047619,329.06,0.0,0.0,0.0,0.0
4,2020-02-05,Alabama,0.0,0,130623.047619,332.86,0.0,0.0,0.0,0.0


## Uploading to Google Sheet

As Tableau Public (the free version) doesn not support a database connection, we will have to use Google Sheet to avoid fragile flat file connections

In [56]:
# Importing Google Sheet Connection Tool

from DataCollection.GoogleSheetUpload import GoogleSheet

In [57]:
# Establishing a connection with the API Key

GoogleSheetBridge = GoogleSheet(key="../DataCollection/SecretKeys/tableaucoviddemo-f9907cac4544.json",
                                url="https://docs.google.com/spreadsheets/d/1no4IXxDRTBqvkm59CGLSLDz7SomSeCbdfpcRaNtQ83I/edit#gid=0")
GoogleSheetBridge.connect()

In [58]:
GoogleSheetBridge.push_to_gsheet(OutputJoin, sheetname="main")

Please head to Tableau Public for the visualization